# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

### Общая информация

Дата выдачи: 28.01.2022

Мягкий дедлайн: 23:59MSK 14.02.2022

Жесткий дедлайн: 23:59MSK 17.02.2022

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимальная оценка за работу (без учёта бонусов) — 10 баллов.

Сдавать задание после указанного жёсткого срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-08-random-features-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Fourier Features (RFF, также в литературе встречается название Random Kitchen Sinks) для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [1]:
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()
x_train = x_train_pics.reshape(x_train_pics.shape[0], -1)
x_test = x_test_pics.reshape(x_test_pics.shape[0], -1)

2022-02-13 22:49:37.567317: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-13 22:49:37.567438: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


__Задание 1. (5 баллов)__

Реализуйте алгоритм, описанный выше. Можете воспользоваться шаблоном класса ниже или написать свой интерфейс.

Ваша реализация должна поддерживать следующие опции:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [115]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import random

def set_random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)


class RFFPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=1000, pca=PCA(n_components=50), ss=StandardScaler(), classifier=LogisticRegression(),
                 cnt_sigma_compute_samples=int(1e6), is_verbose=False):
        """        
        Implements pipeline, which consists of PCA decomposition,
        Random Fourier Features approximation and linear classification model.
        
        n_features, int: amount of synthetic random features generated with RFF approximation.

        ss, StandardScaler: overparametrized StandardScaler, could be None if don't wanted to be used.

        pca, PCA: overparametrized PCA, could be None if don't wanted to be used.
        
        classifier, object: could fit, predict and predict_proba.

        cnt_sigma_compute_samples, int: count of samples needed to calculate sigma^2.

        is_verbose, bool: if True logs out some progress logs.
        
        Feel free to edit this template for your preferences.    
        """
        self.n_features = n_features
        self.pca = pca
        self.is_verbose = is_verbose
        self.classifier = classifier
        self.ss = ss
        self.cnt_sigma_compute_samples = cnt_sigma_compute_samples
        self.w = None
        self.b = None


    def _make_random_samples(self, X):
        samples = np.random.choice(X.shape[0], (self.cnt_sigma_compute_samples, 2))
        samples[samples[:, 0] == samples[:, 1], 1] = (samples[samples[:, 0] == samples[:, 1], 0]\
            + np.random.randint(X.shape[0] - 1, size=samples[samples[:, 0] == samples[:, 1]].shape[0]) + 1) % X.shape[0]
        return samples


    def _compute_sigma2(self, X):
        sigma2_as_vector = np.zeros(self.cnt_sigma_compute_samples)
        samples = self._make_random_samples(X)
        for k, (i, j) in enumerate(samples):
            sigma2_as_vector[k] = ((X[i] - X[j]) ** 2).sum()
        return np.median(sigma2_as_vector)

    def _generate_w_b(self, X):
        sigma2 = self._compute_sigma2(X)
        assert sigma2 > 0, 'sigma^2 cant be less or equal 0'        
        self.w = np.random.normal(0, 1 / np.sqrt(sigma2), (X.shape[1], self.n_features))
        self.b = np.random.rand(self.n_features)* 2 * np.pi - np.pi
        return self.w, self.b

    def _apply_randomized_features(self, X):
        assert self.w is not None and self.b is not None, 'w or b cant be None'
        return np.cos(X.dot(self.w) + self.b)

    def log(self, *args):
        if self.is_verbose:
            print(*args)

    def fit(self, X, y):
        """
        Fit all parts of algorithm (PCA, RFF, Classification) to training set.
        """
        if self.ss is not None:
            self.log('Standart scaller fit_transform')
            X = self.ss.fit_transform(X)
        if self.pca is not None:
            self.log('PCA fit_transform')
            X = self.pca.fit_transform(X)
        self.log('Generating w, b')
        self._generate_w_b(X)
        self.log('Applying w, b')
        X = self._apply_randomized_features(X)
        self.log('Fitting classifier')
        self.classifier.fit(X, y)
        return self

    def _transform_data(self, X):
        """
        Apply data pipeline
        return transformed data
        """
        if self.ss is not None:
            self.log('Standart scaller transform')
            X = self.ss.transform(X)
        if self.pca is not None:
            self.log('PCA transform')
            X = self.pca.transform(X)
        self.log('Applying w, b')
        return self._apply_randomized_features(X)

    def predict_proba(self, X):
        """
        Apply pipeline to obtain scores for input data.
        """
        X = self._transform_data(X)
        self.log('Predicting')
        return self.classifier.predict_proba(X)
        
    def predict(self, X):
        """
        Apply pipeline to obtain discrete predictions for input data.
        """
        X = self._transform_data(X)
        self.log('Predicting')
        return self.classifier.predict(X)

In [116]:
set_random_seed(997)
pipeline = RFFPipeline()
pipeline.fit(x_train, y_train)

/home/panesher/.anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RFFPipeline()

In [117]:
test_predict = pipeline.predict(x_test)
accuracy_score(y_test, test_predict)

0.8552

__Задание 2. (3 балла)__

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучаете градиентный бустинг. Используйте одну из реализаций CatBoost/LightGBM/XGBoost, не забудьте подобрать число деревьев и длину шага.

Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

In [119]:
from sklearn.svm import LinearSVC

set_random_seed(997)
pipeline = RFFPipeline(classifier=LinearSVC())
pipeline.fit(x_train, y_train)
test_predict = pipeline.predict(x_test)
lin_svc_ac_score = accuracy_score(y_test, test_predict)
lin_svc_ac_score

/home/panesher/.anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.8725

In [ ]:
lin_svc_ac_score = 0.8725
lin_svc_ac_score

In [ ]:
from sklearn.svm import SVC

set_random_seed(997)
pipeline = RFFPipeline(classifier=SVC())
pipeline.fit(x_train[:10000], y_train[:10000])
test_predict = pipeline.predict(x_test)
unlin_svc_ac_score = accuracy_score(y_test, test_predict)
unlin_svc_ac_score

In [ ]:
!pip install optuna
!pip install xgboost

In [ ]:
import xgboost as xgb
from IPython.display import clear_output
import optuna

dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_test)

def train_and_test_xgb(trial):
    params = dict()
    params.update({'subsample' : trial.suggest_float('subsample', 0.0001, 1)})
    params.update({'learning_rate' : trial.suggest_float('eta', 0.0001, 1)})
    params.update({'gamma' : trial.suggest_float('gamma', 0, 10)})
    params.update({'max_depth' : trial.suggest_int('max_depth', 1, 50)})

    bst = xgb.train(params, dtrain, trial.suggest_int('n_estimators', 1, 100))
    return accuracy_score(y_test, bst.predict(dvalid))

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(train_and_test_xgb, n_trials=100)
clear_output()

boost_ac_score = study_xgb.best_value
print('Best params:', study_xgb.best_params, '\nResult:', study_xgb.best_value)

In [ ]:
print()

__Задание 3. (2 балла)__

Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA? 
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

### Бонус

__Задание 4. (Максимум 2 балла)__

Как вы, должно быть, помните с курса МО-1, многие алгоритмы машинного обучения работают лучше, если признаки данных некоррелированы. Оказывается, что для RFF существует модификация, позволяющая получать ортогональные случайные признаки (Orthogonal Random Features, ORF). Об этом методе можно прочитать в [статье](https://proceedings.neurips.cc/paper/2016/file/53adaf494dc89ef7196d73636eb2451b-Paper.pdf). Реализуйте класс для вычисления ORF по аналогии с основным заданием. Обратите внимание, что ваш класс должен уметь работать со случаем n_features > new_dim (в статье есть замечание на этот счет). Проведите эксперименты, сравнивающие RFF и ORF, сделайте выводы.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

__Задание 5. (Максимум 2 балла)__

Поэкспериментируйте с функциями для вычисления новых случайных признаков. Не обязательно использовать косинус от скалярного произведения — можно брать знак от него, хэш и т.д. Придумайте побольше вариантов для генерации признаков и проверьте, не получается ли с их помощью добиваться более высокого качества. Также можете попробовать другой классификатор поверх случайных признаков, сравните результаты.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪